## Get yourself familiar with gNMI and IOS XR

In this lab, we will observe how we can manage and configure IOS XR effectively using gNMI (gRPC Network Management Interface). We will use pre-defined gNMI RPCs to automatically configure the IOS XR device and monitor the real-time bgp route count on the device.

All Cisco platforms currently support the gNMI protocol. In the case of IOS XR, the protocol is supported beginning with release 6.5.1.

gNMI is an alternative to network management protocols such as NETCONF and RESTCONF and it is built in the gRPC Remote Procedure Call (gRPC) framework. Therefore, in order to understand gNMI, we first need to understand gRPC.


### gRPC overview

gRPC is an open-source framework that creates efficient and dependable communication in distributed systems. For example, we could use gRPC to connect front-end applications with backend services or to connect management systems with network elements.

It is an implementation of traditional RPC but with some additional features on top of it. Let's see what are those features:

#### gRPC encoding

The payload message transmitted via RPCs can be encoded in protobuf. It should be noted that alternative encoding formats, such as JSON, are also possible.

Protobuf provides a language-agnostic interface description language (IDL) that allows you to define the structure of your data using a simple and concise syntax. The defined data structure is then compiled into code in various programming languages, generating language-specific classes or structures that can be used to serialize and deserialize data.

Given a protobuf file, one can generate bindings in various programming languages for the implementation of the underlying service described in the proto file. Proto files usually also have data structures defined in them which describe messages that are transferred with the RPC calls. Due to this definition, we can avoid hand-parsing the messages that are passed along. All these mean that we can transition from an abstract concept and automatize the utilization of that concept.

<mark>NOTE</mark> - For a reference on the gNMI service definition, see its corresponding protobuf definition at: https://github.com/openconfig/gnmi/blob/master/proto/gnmi/gnmi.proto.


#### gRPC transport

gRPC utilizes the HTTP/2 transport protocol and offers support for Remote Procedure Calls (RPCs). RPCs are operations that can be remotely invoked by the client on the server side, with the inclusion of specific parameters. The server then executes these operations and returns the output as a response to the client. These are the following categories of RPCs:

- Unary RPC: In this type, the client sends a single request to the server and receives a single response in return. It is commonly used for tasks like polling information.

- Server streaming RPC: With this type, the client sends a request to the server to receive multiple streams of information. For instance, it can be used for obtaining model-driven telemetry. The client can also wait for completion, indicated by metadata provided by the server.

- Client streaming RPC: This type involves the client sending multiple streams of information to the server. The server typically responds with a single response.

- Bidirectional streaming RPC: In this category, multiple read-write streams are established between the client and the server, allowing for ongoing streaming communication.


#### gRPC implementation

To establish communication between the server and the client, a gRPC channel is created between the hosts. This requires specifying the IP address and port of the server host. The client has the ability to modify channel traits, such as enabling or disabling message compression, and can monitor the channel's state.

On the server side, there is an implementation of the abstract service that handles RPC calls and encodes the responses. Similarly, the client side also has an implementation of the same service and can invoke methods on the gRPC channel established between the client and the server.

#### gRPC authentication

gRPC supports following authentication mechanism:

- SSL/TLS-based Authentication: Both the client and server are authenticated using digital certificates. This ensures the confidentiality and integrity of the communication channel.

- Token-based Authentication: gRPC allows authentication based on tokens, such as JSON Web Tokens (JWT). The client includes a token in the request headers to authenticate with the server.

- Custom Authentication: gRPC also allows for the implementation of custom authentication methods. Developers can extend the gRPC implementation to integrate their own authentication mechanisms, such as OAuth or custom authentication protocols.


### gNMI

gNMI is dedicated for managing and monitoring the state of network elements. It supports:
- Configuration management and retrieval
- Operational state retrieval
- Bulk data collection via Model-Driven Telemetry (MDT)

In practice, the configuration is performed by the management system (client) that has read-write permissions and is applied to the network element (target). 

The messages within the gRPC service are defined as protocol buffers. The data has to be structured in a tree-like structure, similar to YANG models, and it has to be addressable by paths within that tree.

#### gNMI data payloads and paths

In general, gNMI transports YANG schema instances, but it can also transport data from other types of structures. In any case, a node will be identified by a path within the structure's tree and its value will be serialized.

#### gNMI RPCs

The gNMI RPCs sent between the client and the server are:

- Capabilities RPC - this lists features supported by the network element. In this case, it will list all supported YANG models on the target.
- Get RPC - a unary RPC; it is sent by the client to ask a snapshot of the target's state (configuration / operational).
- Set RPC - a unary RPC; it is sent by the client to update the state of the target (update / delete / replace).
- Subscribe RPC - a bidirectional streaming RPC; the data can be sent as soon as it is available and hence avoids overloading the network element.


This notebook will address four possible RPC calls that can be made with gNMI:
- Retrieval of capabilities of a network element
- Retrieval of state based on CLI format
- Retrieval and configuration of state based on YANG and XPaths
- Configuration of MDT streaming


#### gNMI Clients

Currently there are few clients available on the market for the consumption. Fully open sourced with different level of documentation. Let's take a look at them: 

- OpenConfig gNMI CLI client - official OpenConfig client. Good undestanding of GO required.  
- Google gnxi - one more client from Google. 
- pyGNMI - Python version of gNMI client; 
- gNMIc - Go version of gNMI client - recently transferred to OpenConfig. Created by Nokia
- cisco-gnmi-python - Cisco's version of gNMI client.


For this exercise we will stick with pyGNMI. 




### Preparation of the environment

Install pyGNMI client via PIP: 

In [1]:

pip install pygnmi

Note: you may need to restart the kernel to use updated packages.


Enable logging at the debug level:

In [2]:
#Code block 1

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logging.debug("test")

DEBUG:root:test


Import the Cisco gNMI package and its client module:

In [3]:
# Code block 2
from pygnmi.client import gNMIclient
import json

### Connectivity to the device

Assuming that the device has its gRPC interface configured (`grpc port <port-number> no-tls`), connect to the device using the following details:
- IP address: `198.18.134.1`
- gRPC port:  `<port-number>`
- operating system: `IOS XR`
- username: `cisco`
- password: `cisco123`



In [4]:
target_ip_port = ("198.18.134.1",62204)
target_username = "cisco"
target_password = "cisco123"

### Retrieval of gNMI capabilities

A client can discover the capabilities of its target using the `Capabilities` RPC. The target will reply with a message that contains:
- the version of its gNMI service
- the supported encodings
- the supported data models (YANG models)



In [5]:
# Code block 3
with gNMIclient(target = (target_ip_port), username = target_username, password = target_password, insecure = True) as gc:
    capability_result = gc.capabilities()
    
print(json.dumps(capability_result, indent=4))


INFO:pygnmi.client:Collecting Capabilities...


Collection of Capabilities is successfull


INFO:pygnmi.client:Collection of Capabilities is successfull


{
    "supported_models": [
        {
            "name": "Cisco-IOS-XR-um-macsec-cfg",
            "organization": "Cisco Systems, Inc.",
            "version": "2021-01-05"
        },
        {
            "name": "Cisco-IOS-XR-um-cont-ether-macsec-pm-cfg",
            "organization": "Cisco Systems, Inc.",
            "version": "2021-10-14"
        },
        {
            "name": "openconfig-platform",
            "organization": "OpenConfig working group",
            "version": "0.14.0"
        },
        {
            "name": "openconfig-mpls-sr",
            "organization": "OpenConfig working group",
            "version": "3.0.1"
        },
        {
            "name": "openconfig-igmp-types",
            "organization": "OpenConfig working group",
            "version": "0.1.1"
        },
        {
            "name": "Cisco-IOS-XR-ethernet-link-oam-oper",
            "organization": "Cisco Systems, Inc.",
            "version": "2019-04-05"
        },
        {
          

In the output, we just saw the full list of capabilities (YANG Models). Let's continue to explore and see which specification of gNMI and type of encodings are supported. 

In [6]:
# Code block 4
print(capability_result["supported_encodings"])
print(capability_result["gnmi_version"])

['json_ietf', 'ascii', 'proto']
0.8.0


Voila! We started our gNMI journey. 

### Retrieval and configuration of state information based on YANG and XPaths

<span class="mark">NOTE</span> - References on YANG and JSON encoding for YANG:
  - [RFC 6020 - YANG - A Data Modeling Language for the Network Configuration Protocol (NETCONF)](https://tools.ietf.org/html/rfc6020)
  - [RFC 7950 - The YANG 1.1 Data Modeling Language](https://tools.ietf.org/html/rfc7950)
  - [RFC 7951 - JSON Encoding of Data Modeled with YANG](https://tools.ietf.org/html/rfc7951)

#### YANG - A Data Modeling Language

YANG is a modelling language designed for configuration and description of the state of a system. Its purpose is to enable the manipulation of a system using the Network Configuration Protocol (NETCONF) with NETCONF Remote Procedure Calls (RPCs) and Notifications.

YANG models are defined in highly structured text files and are compilable. YANG's statically-typed syntax and semantic definition ensures the verification of the model's definition at compile time, resulting in errors that can be treated early in the development process.

The models have a tree-based structure with nested sub-elements, named schema nodes. Although they are human readable, the strict formatting and the RFCs defined around them ensure reliability for processing and automation of instance creation of these models.

A network device is said to support a capability if it contains a data model definition of that capability in the form of a module. This module will then contain definitions of data nodes and may import or include definitions from other modules. Such a module is called self-contained and compilable.


#### Snippet of a YANG model
```yang
module Cisco-IOS-XR-telemetry-model-driven-cfg {

  /*** NAMESPACE / PREFIX DEFINITION ***/

  namespace "http://cisco.com/ns/yang"+
    "/Cisco-IOS-XR-telemetry-model-driven-cfg";


  prefix "telemetry-model-driven-cfg";

  /*** LINKAGE (IMPORTS / INCLUDES) ***/

  import ietf-inet-types { prefix "inet"; }

  import Cisco-IOS-XR-types { prefix "xr"; }

  import cisco-semver { prefix "semver"; }
  
  [...]
  
  container telemetry-model-driven {
    status "deprecated";
    description
      "This model is deprecated and is replaced by
      Cisco-IOS-XR-um-telemetry-model-driven-cfg.yang
      which will provide the compatible functionalities
      .  Model Driven Telemetry configuration";

    container sensor-groups {
      description "Sensor group configuration";

      list sensor-group {
        key "sensor-group-identifier";
        description "Sensor group configuration";

        container sensor-paths {
          description "Sensor path configuration";

          list sensor-path {
            key "telemetry-sensor-path";
            description "Sensor path configuration";
            leaf telemetry-sensor-path {
              type string;
              description "Sensor Path";
            }
          }
        }
        leaf sensor-group-identifier {
          type xr:Cisco-ios-xr-string;
          description "The identifier for this group";
        }
      }
    }
```

#### Snippet of a YANG instance encoded in XML
```
<telemetry-model-driven xmlns="http://cisco.com/ns/yang/Cisco-IOS-XR-telemetry-model-driven-cfg">
   <sensor-groups>
    <sensor-group>
     <sensor-group-identifier>S1</sensor-group-identifier>
     <sensor-paths>
      <sensor-path>
       <telemetry-sensor-path>Cisco-IOS-XR-infra-statsd-oper:infra-statistics/interfaces/interface/generic-counters</telemetry-sensor-path>
      </sensor-path>
     </sensor-paths>
    </sensor-group>
   </sensor-groups>
</telemetry-model-driven>
```


#### Snippet of a YANG instance encoded in JSON

```
{
  "Cisco-IOS-XR-telemetry-model-driven-cfg:telemetry-model-driven":  {
     "sensor-groups": {
       "sensor-group": [{
          "sensor-group-identifier": "S1", 
            "sensor-paths": [{
               "sensor-path": {
                   "telemetry-sensor-path":"Cisco-IOS-XR-infra-statsd-oper:infra-statistics/interfaces/interface/generic-counters"
               }
            }]
        }]
     }
}
```

#### XPath

The XML Path Language (XPath) is a language for referencing components in an XML document. Specifically, the most important type in XPath is the LocationPath type that will describe in an expression the absolute or relative location of a node within the hierarchy of the XML tree.

XPath can be used with YANG to address nodes in both the definition and the instantiation of data models. It can further be used in NETCONF and gNMI requests to reference a node or to reference and filter the retrieved data.

<mark>NOTE</mark> - References:
- [The XPath standards](https://www.w3.org/TR/xpath/all/)
- [Whitepaper - XPath in NETCONF and YANG](https://info.tail-f.com/hubfs/Whitepapers/Whitepaper:%20XPath%20in%20NETCONF%20and%20YANG.pdf)


Use the XPath format to reference a YANG structure whose information we want to retrieve. Specifically, we will retrieve the interfaces configured on the IOS XR device. We specify this by first mentioning the **origin** (`Cisco-IOS-XR-um-interface-cfg`), followed by `:`, and then followed by a container or other YANG structure that is part of the YANG model (here, `interfaces`):

In the following code block, we will use gNMI 'GET' RPC to get information for all the interfaces on the router.

In [7]:
# Code block 5
xpath = ["Cisco-IOS-XR-um-interface-cfg:interfaces"]

with gNMIclient(target = (target_ip_port), username = target_username, password = target_password, insecure = True) as gc:
	
	get_config = gc.get(path=xpath, encoding='json_ietf')
    
print(json.dumps(get_config, indent=4))

INFO:pygnmi.client:Collecting info from requested paths (Get operation)...


{
    "notification": [
        {
            "timestamp": 1685844303152574014,
            "prefix": null,
            "alias": null,
            "atomic": false,
            "update": [
                {
                    "path": "interfaces",
                    "val": {
                        "interface": [
                            {
                                "interface-name": "Loopback0",
                                "ipv4": {
                                    "Cisco-IOS-XR-um-if-ip-address-cfg:addresses": {
                                        "address": {
                                            "address": "1.1.1.1",
                                            "netmask": "255.255.255.255"
                                        }
                                    }
                                }
                            },
                            {
                                "interface-name": "MgmtEth0/RP0/CPU0/0",
                        

In the output, we can see there is one 'Loopback0' interface along with other interfaces. We will now create an interface 'Loopback1' using 'SET' RPC of gNMI in the following code block:

In [8]:
# Code block 6
config = [(
  "Cisco-IOS-XR-um-interface-cfg:interfaces/interface",  {
     "interface-name": "Loopback1",
     "ipv4": {
               "Cisco-IOS-XR-um-if-ip-address-cfg:addresses": {
               "address": {
                           "address": "15.15.15.15",
                           "netmask": "255.255.255.255"
                           }
                        }
            }
  })]

with gNMIclient(target = (target_ip_port), username = target_username, password = target_password, insecure = True) as gc:
	set_config = gc.set(update=config, encoding='json_ietf')
    
print(json.dumps(set_config, indent=4))


{
    "timestamp": 1685844314250771591,
    "prefix": null,
    "response": [
        {
            "path": "interfaces/interface",
            "op": "UPDATE"
        }
    ]
}


We'll confirm that new interface was created using gNMI 'GET' RPC in the following code block:

In [12]:
# Code block 7
xpath = ["/Cisco-IOS-XR-um-interface-cfg:interfaces"]

with gNMIclient(target = (target_ip_port), username = target_username, password = target_password, insecure = True) as gc:
	
	get_config = gc.get(path=xpath, encoding='json_ietf')
print(json.dumps(get_config, indent=4))

INFO:pygnmi.client:Collecting info from requested paths (Get operation)...


{
    "notification": [
        {
            "timestamp": 1685844530652119732,
            "prefix": null,
            "alias": null,
            "atomic": false,
            "update": [
                {
                    "path": "interfaces",
                    "val": {
                        "interface": [
                            {
                                "interface-name": "Loopback0",
                                "ipv4": {
                                    "Cisco-IOS-XR-um-if-ip-address-cfg:addresses": {
                                        "address": {
                                            "address": "1.1.1.1",
                                            "netmask": "255.255.255.255"
                                        }
                                    }
                                }
                            },
                            {
                                "interface-name": "Loopback1",
                                "i

And we can see that an interface 'Loopback1' is created on the router. 

### Streaming telemetry based on YANG and XPaths

We will now use gNMI 'SUBSCRIBE' RPC to stream telemetry in every 10 seconds  from the router. Here, we will stream number of BGP routes learned by the router.



In [13]:
# Code block 

subscribe_request = {
	"subscription":[
		{
			"path":"Cisco-IOS-XR-ipv4-bgp-oc-oper:oc-bgp/bgp-rib/afi-safi-table/ipv4-unicast/open-config-neighbors/open-config-neighbor/adj-rib-in-post/num-routes/num-routes",
			"mode":"sample",
			"sample_interval":10000000000 ## in nano seconds
		}],
		"mode":"stream",
		"encoding":"json_ietf"
	}

with gNMIclient(target = (target_ip_port), username = target_username, password = target_password, insecure = True) as gc:
	telemetry_stream = gc.subscribe(subscribe=subscribe_request)

	for data in telemetry_stream:
			print(data)
            
            

INFO:pygnmi.client:Collecting Telemetry...


update {
  timestamp: 1685844536658000000
  prefix {
    origin: "Cisco-IOS-XR-ipv4-bgp-oc-oper"
  }
  update {
    path {
      elem {
        name: "oc-bgp"
      }
      elem {
        name: "bgp-rib"
      }
      elem {
        name: "afi-safi-table"
      }
      elem {
        name: "ipv4-unicast"
      }
      elem {
        name: "open-config-neighbors"
      }
      elem {
        name: "open-config-neighbor"
        key {
          key: "neighbor-address"
          value: "192.168.122.1"
        }
      }
      elem {
        name: "adj-rib-in-post"
      }
      elem {
        name: "num-routes"
      }
    }
    val {
      json_ietf_val: "{\"num-routes\":\"10000\"}"
    }
  }
}

sync_response: true

update {
  timestamp: 1685844546656000000
  prefix {
    origin: "Cisco-IOS-XR-ipv4-bgp-oc-oper"
  }
  update {
    path {
      elem {
        name: "oc-bgp"
      }
      elem {
        name: "bgp-rib"
      }
      elem {
        name: "afi-safi-table"
      }
      elem {


KeyboardInterrupt: 

### Configuration of model-driven telemetry streaming (persistent, dial-out)

Configure the IOS XR device for model-driven telemetry streaming using the **gRPC dial-out** mechanism. In this case, the device initiates a gRPC session (**dial-out**) with the MDT receiver and exchanges SYN -- SYN-ACK -- ACK with it while establishing the connection.

If the connection is successfully created, the device will start streaming telemetry data towards the collection point.

The streaming job will run indefinitely on the IOS XR device, even if the receiver is unreachable. The device will repeatedly try to contact the receiver, every 30 seconds.

<mark>NOTE</mark> - The dial-out mechanism is supported in both TCP and gRPC.

A dial-out MDT streaming configuration is **persistent** in the configuration of the IOS XR device and will be consistent throughout device reboots. It consists of:
- A **sensor-group** that is composed of sensor-path(s):
 - A sensor-path is a specification of the YANG component about which information should be streamed; specified in XPath format
- A **destination-group** that is composed of destination(s):
 - A destination contains the receiver's specifications:
   - IP address
   - port
   - encoding format
   - protocol
- A **subscription** that brings together a sensor-group, a destination-group and a time interval [ms]

Here, we subscribe to updates of the operational state (`Cisco-IOS-XR-ipv4-bgp-oc-oper`). Specifically, we are looking for number of BGP routes:

Now, we will check current MDT configuration on the router.

In [6]:
 
xpath = ["Cisco-IOS-XR-telemetry-model-driven-cfg:telemetry-model-driven"]

with gNMIclient(target = (target_ip_port), username = target_username, password = target_password, insecure = True) as gc:
	
	get_config = gc.get(path=xpath, encoding='json_ietf')
print(json.dumps(get_config, indent=4))

INFO:pygnmi.client:Collecting info from requested paths (Get operation)...


{
    "notification": [
        {
            "timestamp": 1685859650343408723,
            "prefix": null,
            "alias": null,
            "atomic": false,
            "update": [
                {
                    "path": "telemetry-model-driven",
                    "val": {
                        "destination-groups": {
                            "destination-group": [
                                {
                                    "destination-id": "D1_gnmi",
                                    "ipv4-destinations": {
                                        "ipv4-destination": [
                                            {
                                                "ipv4-address": "192.168.122.1",
                                                "destination-port": 57101,
                                                "encoding": "self-describing-gpb",
                                                "protocol": {
                                             

We will now create subscription for MDT

In [7]:
# Code block 
config = [(
  "Cisco-IOS-XR-telemetry-model-driven-cfg:telemetry-model-driven",  {
    "subscriptions": {
        "subscription": [{
            "sensor-profiles": {
                "sensor-profile": [{
                    "sensorgroupid": "S1_gnmi",
                    "sample-interval": 10000
                }]
            },
            "destination-profiles": {
                "destination-profile": [{
                    "destination-id": "D1_gnmi"
                }]
            },
            "subscription-identifier": "Sub1_gnmi"
        }]
    }
  }
)]

with gNMIclient(target = (target_ip_port), username = target_username, password = target_password, insecure = True) as gc:
	set_config = gc.set(update=config, encoding='json_ietf')
    
print(json.dumps(set_config, indent=4))
    
    

{
    "timestamp": 1685859664322982327,
    "prefix": null,
    "response": [
        {
            "path": "telemetry-model-driven",
            "op": "UPDATE"
        }
    ]
}



Confirm that the new MDT **configuration** has been saved on the device:

In [8]:
# Code block 

xpath = ['Cisco-IOS-XR-telemetry-model-driven-cfg:telemetry-model-driven']

with gNMIclient(target = (target_ip_port), username = target_username, password = target_password, insecure = True) as gc:
	
	get_config = gc.get(path=xpath, encoding='json_ietf')
print(json.dumps(get_config, indent=4))

INFO:pygnmi.client:Collecting info from requested paths (Get operation)...


{
    "notification": [
        {
            "timestamp": 1685859667837566072,
            "prefix": null,
            "alias": null,
            "atomic": false,
            "update": [
                {
                    "path": "telemetry-model-driven",
                    "val": {
                        "destination-groups": {
                            "destination-group": [
                                {
                                    "destination-id": "D1_gnmi",
                                    "ipv4-destinations": {
                                        "ipv4-destination": [
                                            {
                                                "ipv4-address": "192.168.122.1",
                                                "destination-port": 57101,
                                                "encoding": "self-describing-gpb",
                                                "protocol": {
                                             

Retrieve the **operational state**  of the telemetry streaming subscription. Notice that the subscription is *active*:

In [11]:
# Code block 

xpath = ['Cisco-IOS-XR-telemetry-model-driven-oper:telemetry-model-driven']

with gNMIclient(target = (target_ip_port), username = target_username, password = target_password, insecure = True) as gc:
	
	get_config = gc.get(path=xpath, encoding='json_ietf')
print(json.dumps(get_config, indent=4))


INFO:pygnmi.client:Collecting info from requested paths (Get operation)...


{
    "notification": [
        {
            "timestamp": 1685859710222426646,
            "prefix": null,
            "alias": null,
            "atomic": false,
            "update": [
                {
                    "path": "telemetry-model-driven",
                    "val": {
                        "destinations": {
                            "destination": [
                                {
                                    "destination-id": "D1_gnmi",
                                    "id": "D1_gnmi",
                                    "configured": 1,
                                    "destination": [
                                        {
                                            "destination": {
                                                "id": "2002",
                                                "dest-ip-address": {
                                                    "ip-type": "ipv4",
                                                    "ipv4-add

The host 198.18.134.1 already has the collection components for ingesting, storing and visualizing telemetry data. We are using the following set of Docker containers:
- Telegraf - a collection container that runs the cisco_telemetry_mdt plugin for ingestion of model-driven telemetry data
- InfluxDB - a storage container that runs a database which stores the data
- Grafana - a visualization container that runs a web application which allows the exploration of data

The collector listens on port <port-number> for incoming connections. IOS XR will initiate the connection, dialling out to the collector. After the connection establishment, the device will begin to stream data over gRPC to the collector which will send it to a database named telemetry.

<mark>NOTE</mark> - Please refer to the following references for details about the three components and the ingestion of telemetry data:
- [InfluxData](https://www.influxdata.com/)
- [Input plugin for Model-driven telemetry over gRPC](https://github.com/influxdata/telegraf/tree/master/plugins/inputs/cisco_telemetry_mdt)
- [Input plugin for Model-driven telemetry over gNMI](https://github.com/influxdata/telegraf/tree/master/plugins/inputs/cisco_telemetry_gnmi)


In a separate browser tab, open the web application for data visualization located at http://10.30.110.22:. Then, hover over the left hand-side and click on Dashboards. Open the `Traffic Monitoring` dashboard.

<mark>NOTE</mark> - If you are missing the dashboard, use the `Import Dashboard` button to upload the dashboard located at `monitoring/chronograf/Traffic Monitoring.json`.

<img src="images/traffic.png">

We just tried dial out mechanism. Dial-in mechanism very similar in nature. 

In [29]:
# Code block 

delete_config = ["Cisco-IOS-XR-telemetry-model-driven-cfg:telemetry-model-driven"]

with gNMIclient(target = (target_ip_port), username = target_username, password = target_password, insecure = True) as gc:
	set_config = gc.set(delete=delete_config, encoding='json_ietf')
    
print(json.dumps(set_config, indent=4))

{
    "timestamp": 1685845901152705480,
    "prefix": null,
    "response": [
        {
            "path": "telemetry-model-driven",
            "op": "DELETE"
        }
    ]
}



Confirm that the configuration has been deleted:

In [46]:
# Code block 

xpath = ['Cisco-IOS-XR-telemetry-model-driven-cfg:telemetry-model-driven']

with gNMIclient(target = (target_ip_port), username = target_username, password = target_password, insecure = True) as gc:
	
	get_config = gc.get(path=xpath, encoding='ascii')
print(json.dumps(get_config, indent=4))


INFO:pygnmi.client:Collecting info from requested paths (Get operation)...


{
    "notification": [
        {
            "timestamp": 1685724148731164744,
            "prefix": null,
            "alias": null,
            "atomic": false,
            "update": [
                {
                    "path": "telemetry-model-driven",
                    "val": "\n"
                }
            ]
        }
    ]
}


**This project was originally written by the following individuals:**

- cprecup cprecup@cisco.com
- nckbgov nbelchug@cisco.com
- sbyx stbarth@cisco.com

Updated to pygnmi by mkorshun@cisco.com